In [ ]:

##Import Statements
import tkinter as tk
from tkinter import messagebox
import PyPDF2
import openpyxl
from openpyxl import load_workbook
import shutil
from PyPDF2 import PdfReader
import os
import pandas as pds
newdf=pds.DataFrame()

#GUI Interface
root=tk.Tk()
root.title("DHL_Invoice_Reader")

label=tk.Label(root,text="DHL Invoice Reader",font=("Arial",15,"bold"),bg="grey",fg="black",relief=tk.FLAT,justify=tk.CENTER)
label.pack(padx=25,pady=25)

def button_clicked():

    d2df=pds.DataFrame()
    if len(os.listdir('U:/source/'))>0:
        files=os.fsencode("U:/source")
        for file in os.listdir(files):
            filename=os.fsdecode(file)
            if filename.endswith(".pdf"):
                path='U:/source/'+filename
                reader=PdfReader(path)
                #Data Wrangling
                pages=reader.pages[0]
                text=pages.extract_text().split("\n")
                data1=str(text[18])+str(text[19])+str(text[20])
                data2=data1.split(":")

                res2=[]
                for i in data2:
                      data2=i.replace('\xa0',"#")
                      res2.append(data2)

                res3=[]
                for j in res2:
                     res2=j.replace('#',"")
                     res3.append(res2)

                re1=res3[1][:10]
                re2=res3[1][10:]
                re3=res3[2][:11]
                re4=res3[2][11:]

                res3.pop(1)
                res3.pop(1)


                res3.insert(1,re1)
                res3.insert(2,re2)
                res3.insert(3,re3)
                res3.insert(4,re4)

                r1=[]
                r2=[]
                for i in range(len(res3)):
                   if i%2==0:
                        r1.append(res3[i])
                   else:
                        r2.append(res3[i])

                data3=text[24]
                data4=data3.split(" ")

                var=data4[:4]
                var1=data4[4:]
                var1[1:]=[" ".join(var1[1:])]

                res4=[]
                for item in var1:
                   item=item.replace(":","")
                   res4.append(item)

                for i in range(2):
                    if i%2==0:
                       r1.append(res4[i])
                    else:
                       r2.append(res4[i])

                res=[]

                for i in var:
                    i=i.replace("':","").replace("',","").replace("'","").replace("[","").replace(":","")
                    res.append(i)
                data5=text[25]

                data6=data5.split(" ")
                data6=data6[:4]
                res6=[]
                for item in data6:
                   item=item.replace(":","")
                   res6.append(item)

                r3=[]
                r4=[]
                for i in range(4):
                   if i%2==0:
                      r3.append(res6[i])
                   else:
                      r4.append(res6[i])

                for i in range (4):
                     if i%2==0:
                        r3.append(res[i])
                     else:
                        r4.append(res[i])
                r3.insert(0,r3[2])
                r3.pop(3)

                r4.insert(0,r4[2])
                r4.pop(3)
                d1=dict(zip(r1,r2))

                d2=dict(zip(r3,r4))
                d1.update(d2)
                d1df=pds.DataFrame(d1,index=[0])

                d2df=d2df.append(d1df,ignore_index=True)
                print("d2df",d2df)

                dpath='U:/destination/'
                dest_path=f'{dpath}{filename}'
                shutil.move(path,dest_path)

        e_path='U:/DHL_Invoice_Details.xlsx'
        existing_file=pds.read_excel(e_path)
        book=load_workbook(e_path)
        writer=pds.ExcelWriter(e_path,engine='openpyxl')
        writer.book=book

        #Appending Data to the Excel file

        writer.sheets=dict((ws.title,ws) for ws in book.worksheets)
        d2df.to_excel(writer,sheet_name="DHL_Invoice",index=False,header=False,startrow=writer.sheets['DHL_Invoice'].max_row)
        writer.save()

        messagebox.showinfo(title="Message",message="Successfully updated the Invoice details to Excel File 'DHL_Invoice Reader'.Location: 'U:/DHL_Invoice_Details.xlsx'")

    else:
        messagebox.showerror(title="Warning",message="Your Source File is Empty...!!!")

# Button to trigger the process
button=tk.Button(root,text="Click Me",bg="grey",fg="black",highlightthickness=2,relief=tk.RAISED,command=button_clicked)
button.pack(padx=15,pady=15)
root.mainloop()